In [58]:
# from .utils import *
# from .timedistance import *
# from .delay import *
# from .route import *
from math import sin, cos, sqrt, atan2, radians,inf
from queue import PriorityQueue
from math import inf
import networkx as nx

from utils import *
from algo import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# neighbours()

In [ ]:
def stochastic_dijkstra(self, 
                            start: Union[Name,ID],     # start node (either name or id. If id, names_format to False)
                            end: Union[Name,ID],       # end node (either name or id. If id, names_format to False)
                            arr_time:Union[str,Time],  # target arrival time (ex '12:45:00' or timestamp)
                            threshold = 0.8,           # target probability of making all connections in time
                            names_format=True          # If we want to use IDs for start and end set to False
                           ) -> Optional[Route]:    

    prev = {}                                                                   # predecessor of current node on shortest path 
    distances = {v: TimeDistance(time=arr_time) for v in nx.nodes(self.G)}      # initialize distances from end to any given node
    visited = set()                                                             # nodes we've visited
    # prioritize nodes from start -> node with the shortest distance!
    queue = PriorityQueue()                        
    distances[end].updated()                                                    # dist from end -> end is zero
    queue.put((distances[end], end))                                               # add end node for exploration
    # main search phase of algorithm
    while not queue.empty():
        _, curr = queue.get()
        visited.add(curr)
        for _,neighbor,properties in neighbours(curr):              # look at curr's adjacent nodes
            new_dist = distances[curr] + properties                 # if we found a shorter path 
            if new_dist < distances[neighbor]:                      # update the distance, we found a shorter one!
                distances[neighbor] = new_dist.updated()            # update the previous node to be prev on new shortest path
                prev[neighbor] = curr                               # if we haven't visited the neighbor
                if neighbor not in visited:                         # insert into priority queue and mark as visited
                    visited.add(neighbor)
                    queue.put((distances[neighbor],neighbor))          # prioritize search on small distances
    # compute the route 
    return build_route()

In [ ]:
def build_route() -> Optional[Route]:
    """
    Get the shortest path of nodes by going backwards through prev list
    """
    if start not in prev.keys():
        return None
    
    route = Route(self.G)
    node = start
    dist = distances[start]
    path = []
    while node != end:
        route.append(node,dist)
        node = prev[node]
        dist = distances[node]
    route.append(node,dist) 
    return route

In [60]:
def build_route(prev_trip_id, prev, distances, start_id, end_id):

    if start_id not in prev.keys():
        return None
    
    node = start_id
    # prev_id = prev[start_id]
    trip_id = prev_trip_id[node]
    dist = distances[start_id]
    path = []
    while node != end_id:
        path.append((trip_id, node, dist))
        node = prev[node]
        trip_id = prev_trip_id[node]
        dist = distances[node]
    # route.append(node,dist)
    path.append((trip_id, node, dist))
    return path

In [59]:
end_id = ZURICH_HB_ID
end_time = Time(h=10).in_seconds()


distances = {}      # stores travel_times
prev = {}           # stores predecessor
prev_trip_id = {}   # stores prev_trip_id
# prev_travel_times = {}

visited = set()     # stores already visited stop_ids
queue = PriorityQueue()

distances[end_id] = 0
prev[end_id] = None
prev_trip_id[end_id] = None
# prev_travel_times[end_id] = 0

queue.put((distances[end_id], (end_id, end_time)))

# TODO? Add thing to stop once we reach target?
while not queue.empty():
    _, (curr_id, curr_time) = queue.get()
    visited.add(curr_id)
    for trip_id, neighbor_id, neighbor_dep_time_s in neighbors(curr_id, curr_time, prev_trip_id[curr_id]):
        new_dist = end_time - neighbor_dep_time_s
        # new_dist = distances[curr_id] + neighbor_weight
        if new_dist < distances.get(neighbor_id, inf):

            distances[neighbor_id] = new_dist
            prev[neighbor_id] = curr_id
            prev_trip_id[neighbor_id] = trip_id

            if neighbor_id not in visited:
                visited.add(neighbor_id)
                # prev_dep_time
                queue.put((distances[neighbor_id], (neighbor_id, neighbor_dep_time_s)))



In [61]:
distances

{8503090: 0,
 8503088: 120.0,
 8503051: 240.0,
 8503091: 697.0,
 8591064: 331,
 8591367: 480,
 8591369: 378,
 8591381: 394,
 8591427: 562,
 8503000: 370.0,
 8503446: 538.0,
 8503499: 679.0,
 8587348: 260.0,
 8587349: 360,
 8588078: 480,
 8591067: 396.0,
 8591174: 795.0,
 8591262: 480,
 8591316: 540,
 8591327: 802.0,
 8591368: 595.0,
 8591379: 736.0,
 8596126: 753.0,
 8591245: 909.0,
 8591269: 828.0,
 8591308: 688.0,
 8591366: 720.0,
 8591824: 386.0,
 8591404: 480,
 8591384: 480,
 8503011: 660,
 8503020: 660,
 8503003: 780,
 8591299: 540,
 8591184: 480,
 8591079: 600,
 8591287: 900.0,
 8503500: 900,
 8591123: 1020,
 8591052: 540.0,
 8591277: 660.0,
 8591227: 780.0,
 8591218: 780.0,
 8591219: 840.0,
 8503651: 1185,
 8591093: 600.0,
 8591105: 1117,
 8591317: 900.0,
 8591059: 540.0,
 8591058: 660.0,
 8503010: 720.0,
 8507183: 930.0,
 8591169: 660.0,
 8591415: 600.0,
 8591365: 1200.0,
 8591283: 780.0,
 8591414: 780.0,
 8503447: 985,
 8530822: 1212,
 8591183: 1059,
 8591309: 900,
 8573710: 7

In [62]:
route=build_route(prev_trip_id, prev, distances, 8591192, ZURICH_HB_ID)
route

[('692.TA.26-33-B-j19-1.2.H', 8591192, 1800.0),
 ('692.TA.26-33-B-j19-1.2.H', 8591376, 1740),
 ('walk', 8591411, 1516),
 ('450.TA.26-7-B-j19-1.1.H', 8591373, 900),
 ('walk', 8591174, 795.0),
 ('82.TA.26-4-A-j19-1.4.R', 8503088, 120.0),
 (None, 8503090, 0)]

In [52]:

def build_route_df(route, end_time):
    df_route = pd.DataFrame(route)
    df_route.rename({0:'trip_id', 1:'dep_stop_id', 2:'dep_time'}, axis=1, inplace=True)
    # df_route['dep_time'] = end_time - df_route['dep_time']
    df_route.sort_values(by='dep_time', ascending=True, inplace=True)
    return df_route

build_route_df(route, end_time)

,trip_id,dep_stop_id,dep_time
5,None,8503090,0.0
4,82.TA.26-4-A-j19-1.4.R,8503088,120.0
3,walk,8591174,795.0
2,450.TA.26-7-B-j19-1.1.H,8591373,900.0
1,walk,8591411,1516.0
0,692.TA.26-33-B-j19-1.2.H,8591376,1740.0
